### UX RNN Price Baseline Program (inspired by Jose Padilla's course)

In [ ]:
# Import common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Run matplotlib inline 
%matplotlib inline

In [ ]:
# Import data
df = pd.read_csv('UX-dailyhistory_edited.csv',sep=',', index_col='Date')

In [ ]:
df.head()

In [ ]:
df[-5:]

In [ ]:
df[-5:-1]

In [ ]:
# Define features and target
#features =[
# 'Open'
#,'Max'
#,'Min'
#,'Traded Volume'
#]
target   = ['Close']

X = df[target]

In [ ]:
X.info()

### Train Test Split

* For time series perform train test split based upon sequential order. 

In [ ]:
X_train = X.head(2245)

In [ ]:
X_test = X.tail(5)

### Scale the Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
train_scaled = scaler.fit_transform(X_train)

In [ ]:
test_scaled = scaler.transform(X_test)

### Batch Function

In [ ]:
### Creating a batch function

def next_batch(training_data,steps,batch_size):
    
    # set random starting point index with same number of steps
    rand_start = np.random.randint(len(training_data) - steps)
    
    # create y batch from random start to number of steps plus 1
    y_batch = np.array(training_data[rand_start:rand_start + steps + 1]).reshape(1,steps+1)

    # return y_batch
    return y_batch[:,:-1].reshape(-1,steps,1),y_batch[:,1:].reshape(-1,steps,1)

### Setting Up The RNN Model

** Import TensorFlow **

In [ ]:
import tensorflow as tf

### The Constants

In [ ]:
# Just one feature, the time series
num_inputs = 1

# Num of steps in each batch
num_time_steps = 5

# 100 neuron layer, play with this
num_neurons = 2000

# Just one output, predicted time series
num_outputs = 1

## You can also try increasing iterations, but decreasing learning rate
# learning rate you can play with this
learning_rate = 0.001

# how many iterations to go through (training steps), you can play with this
num_train_iterations = 2000

# Size of the batch of data
batch_size = 1


In [ ]:
test_scaled.shape

In [ ]:
X = tf.placeholder(tf.float32, [None, num_time_steps, num_inputs])
y = tf.placeholder(tf.float32, [None, num_time_steps, num_outputs])

In [ ]:
# Also play around with GRUCell
lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons,activation=tf.nn.relu),
    output_size=num_outputs)

In [ ]:
lstm_outputs, lstm_states = tf.nn.dynamic_rnn(lstm_cell,X,dtype=tf.float32)

In [ ]:
loss = tf.reduce_mean(tf.square(lstm_outputs - y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_err = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)
    
    for iteration in range(num_train_iterations):
        
        x_batch, y_batch = next_batch(train_scaled,num_time_steps,batch_size)
        
        sess.run(train_err,feed_dict={X:x_batch,y:y_batch})
        
        if iteration%100 == 0:
            
            mse = loss.eval(feed_dict={X:x_batch,y:y_batch})
            
            print(iteration,'MSE is:',mse)
    
          
    saver.save(sess,'./stock_model')

In [ ]:
with tf.Session() as sess:
    
    saver.restore(sess,"./stock_model")
    
    train_seed = list(test_scaled[-5:]) #scaled data without last 5 days, entire training set
    
    for iterations in range(5):
        
        X_batch = np.array(train_seed[-num_time_steps:]).reshape(1,num_time_steps,1)
        
        y_pred = sess.run(lstm_outputs,feed_dict={X:X_batch})
        
        train_seed.append(y_pred[0,-1,0]) #append y predicted values to training seed

In [ ]:
predictions = scaler.inverse_transform(np.array(train_seed[5:]).reshape(5,1))

In [ ]:
X_test['Generated'] = predictions

In [ ]:
X_test

In [ ]:
X_test.plot()

In [ ]:
X_train[-4:] #test

In [ ]:
X_train[-5:-1] #train

In [ ]:
X_test['Close']

In [ ]:
plt.title("Testing Model")

# Training Instance
plt.plot(X_train[-10:-1], "bo", markersize=25,alpha=0.5, label="Training Instance")

# Target to Predict
plt.plot(X_train[-10:], "ko", markersize=10, label="Testing Instance")

# Models Prediction
plt.plot(X_test['Generated'], "r.", markersize=10, label="Prediction")

plt.ylabel("UX Price")
plt.legend()
plt.tight_layout()